# The Appliance of Science

SciPy (the Scientific Python library) is a collection of mathematical algorithms and convenience functions built on the NumPy extension of Python. This notebook will focus on applying numerical methods.

The [docs](https://docs.scipy.org/doc/scipy/reference/index.html) should be consulted for extra info.

It is organised into subpackages covering different themes, and are (by convention) individually imported. Below are a few examples:

|Package          |Description                        |
|:----------------|:----------------------------------|
|scipy.constants  |Physical and mathematical constants|
|scipy.fftpack    |Fourier transform                  |
|scipy.integrate  |Integration routines               |
|scipy.interpolate|Interpolation                      |
|scipy.linalg     |Linear algebra routines            |
|scipy.optimize   |Optimization                       |
|scipy.signal     |Signal processing                  |
|scipy.special    |Any special mathematical functions |
|scipy.stats      |Statistics                         |

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scientific-constants" data-toc-modified-id="Scientific-constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scientific constants</a></span></li><li><span><a href="#Special-functions" data-toc-modified-id="Special-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Special functions</a></span></li><li><span><a href="#Nonlinear-root-finding" data-toc-modified-id="Nonlinear-root-finding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Nonlinear root-finding</a></span></li><li><span><a href="#Nonpolynomial-curve-fitting" data-toc-modified-id="Nonpolynomial-curve-fitting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Nonpolynomial curve fitting</a></span></li><li><span><a href="#Interpolation" data-toc-modified-id="Interpolation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interpolation</a></span></li><li><span><a href="#Numerical-integration" data-toc-modified-id="Numerical-integration-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Numerical integration</a></span></li><li><span><a href="#Solving-ordinary-diﬀerential-equations" data-toc-modified-id="Solving-ordinary-diﬀerential-equations-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Solving ordinary diﬀerential equations</a></span></li><li><span><a href="#Introduction-to-optimisation" data-toc-modified-id="Introduction-to-optimisation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Introduction to optimisation</a></span></li></ul></div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
%matplotlib inline

In [ ]:
# scipy?

## Scientific constants

Physical constants and unit conversion factors can either be accessed by importing the subpackage and calling them as methods, or by passing a very specific keyword argument to the `physical_constants` dictionary. The latter will return the value, unit, and uncertainty. The methods and keywords can be found [here](https://docs.scipy.org/doc/scipy-1.2.0/reference/constants.html).

In [ ]:
from scipy import constants as con

In [ ]:
print(con.R)   # molar gas constant
print(con.N_A) # Avogadro constant
print(con.k)   # Boltzmann constant
print(con.m_e) # electron mass

In [ ]:
# Conversion factors
print(con.kilogram_force)
print(con.atm)
print(con.degree_Fahrenheit)
print(con.psi)

In [ ]:
con.physical_constants['atomic unit of mass']

In [ ]:
con.physical_constants['molar volume of ideal gas (273.15 K, 100 kPa)']

## Special functions

Sometimes very involved mathematics are needed to solve problems, for instance, the Bessel functions ($j_0$ & $j_1$) and error function in transient heat transfer problems. 

In [ ]:
# help(scipy.special)

In [ ]:
from scipy.special import j0, j1, erf

In [ ]:
# Let's evaluate them at 0.1
print(j0(0.1))
print(j1(0.1))

In [ ]:
xspan = np.linspace(-3, 3, 100)
plt.plot(xspan, erf(xspan))
plt.show()

## Nonlinear root-finding

Sometimes we need to find an $ x $ such that $ f(x) = 0 $, especially for non-linear or implicit functions. There are 2 ways to do this: `fsolve()` which only needs one starting point close to the root (but is not guaranteed to converge); or `bisect()` which is more robust, but slower and requires an interval. Both of these methods also work for systems of equations. In any case, it's usually a good idea to plot the function to see where roots may be. 

Let's consider the Colebrook equation for the friction factor $f$. We'll choose the parameters as roughness (ϵ) 0.046 mm, diameter (D) 20 mm, and Reynolds number (Re) 200 000:

$$ \frac{1}{\sqrt{f}} = -2\log_{10} \left( \frac{\epsilon/D}{3.7} + \frac{2.51}{\mathrm{Re} \sqrt{f}}\right)  $$

<span style="color:red"> **Warning:** </span> We must rewrite the function such that it equals zero. If a poor guess value for the root is given, it may not converge, or find the wrong root; therefore, always try to plot the function to see where the root may lie.

In [ ]:
from scipy.optimize import fsolve, bisect

In [ ]:
def colebrook(f, const):
    ϵ, D, Re = const
    return 1/f**0.5 + 2*np.log10(ϵ/D/3.7 + 2.51/(Re*f**0.5))

param = [0.046, 20, 200000]
fspan = np.linspace(0.01, 0.08, 100)

plt.plot(fspan, colebrook(fspan, param))
plt.plot([fspan[0], fspan[-1]], [0, 0], '--')
plt.show()

From the graph we can see that the root of the Colebrook equation lies between 0.02 and 0.03.

In [ ]:
fsolve(colebrook, 0.03, param)

In [ ]:
bisect(colebrook, 0.02, 0.03, param)

## Nonpolynomial curve fitting

Recall from Unit 3, that we can fit polynomials to data. However, for nonpolynomial fits we'll have to use `curve_fit`. To illustrate, let's use the permeability data of sample A3 in the Experimental data file.

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# curve_fit?

In [ ]:
df_perm = pd.read_excel('Assets/Experimental data.xlsx', sheet_name='Permeability')
time = df_perm['t_cum']
A3 = df_perm['A3']/30*100

In [ ]:
plt.plot(time, A3, 'o', label='A3')
plt.ylabel('Cummulative mass loss (%)')
plt.xlabel('Time (h)')
plt.legend()
plt.show()

You'll have to be creative to come up with a function that describes your data. Since we can see an exponential trend, let's try
$$ m(t) = a + b(1-e^{-c t}) $$

We'll guess values for the three parameters ($a, b,$ and $c$) and use the `*args` notation to pass them as positional arguments to the function (revise Unit 2 if needed). This is especially useful if you are continuously changing the number of parameters, as it enables passing an "unknown" number of arguments.

In [ ]:
def func(t, a, b, c):
    return a + b*(1 - np.exp(-c*t))

tvals = np.linspace(0, 400, 400)
guess = [0.15, 0.05, 0.05]

plt.plot(time, A3, 'o', label='A3')
plt.plot(tvals, func(tvals, *guess))
plt.ylabel('m (%)')
plt.xlabel('t (h)')
plt.show()

We can see that the shape is more or less correct, but the parameter values are far off. Now we can find the optimal parameter values and the estimated covariance matrix (usually ignored).

In [ ]:
popt, pcov = curve_fit(func, time, A3, guess)
popt

We can now plot the trend line. We can use the `format` attribute from Unit 1 to pass the parameter values to the legend label. Note that the exponent argument must be enclosed by double braces to differentiate $\LaTeX$ syntax from the formatted printing (see Unit 8).

In [ ]:
plt.plot(time, A3, 'o')
plt.plot(tvals, func(tvals, *popt), label='$m = {:.3f}+{:.3f}(1-e^{{-{:.3f}t}})$'.format(*popt))
plt.legend()
plt.ylabel('m (%)')
plt.xlabel('t (h)')
plt.show()

## Interpolation

Sometimes instead of relying on a fitted equation, you want to interpolate between your data points. You can create functions based on your data that can interpolate linearly, quadratically, or otherwise. 

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
# interp1d?

In [ ]:
x = time
y = A3
f_lin = interp1d(x, y)
f_cub = interp1d(x, y, kind='quadratic')

print(f_lin(25))
print(f_cub(25))

xnew = np.arange(0, 379, 5)

plt.plot(x, y, 'o')
plt.plot(xnew, f_lin(xnew), '-', label='linear')
plt.plot(xnew, f_cub(xnew), '--', label='quadratic')
plt.legend()
plt.show()

## Numerical integration

Numerical integration (aka quadrature) is possible by several routines. Only the `quad` routines allow functions as input, whereas the others require data points:
- `quad`: Compute a definite integral
- `dblquad`: Compute a double integral.
- `tplquad`: Compute a triple integral. 
- `trapz`: Integrate along the given axis using the composite trapezoidal rule.
- `cumtrapz`: Cumulatively integrate y using the composite trapezoidal rule (i.e. outputs an array).
- `simps`: Integrate along the given axis using Simpson's rule.

Let's numerically evaluate $$ \int^2_{-2} x^2 \, \mathrm{d}x = \frac{16}{3} $$

In [ ]:
from scipy.integrate import quad, trapz, simps 

In [ ]:
def f(x):
    return x**2

# It returns the value, and an estimate of the absolute error in the result.
quad(f, -2, 2)

In [ ]:
x = np.linspace(-2, 2, num=100)
y = x**2

print(trapz(y, x))
print(simps(y, x))

The error of the trapezoidal rule is proportional to $ h^2[f'(a) - f'(b)]$ where $h$ is the step size. For Simpson's rule it is proportional to $ h^4[f'''(a) - f'''(b)]$. Thus, for "smooth" functions it is generally better to use Simpson's rule; however, for periodic and peak functions, the trapezoidal rule may converge faster.

## Solving ordinary diﬀerential equations

To solve a single ODE or a set of ODEs, use `solve_ivp()` to solve a differential equaton. Note that in older code, you may encounter `odeint` which has been deprecated.

In [ ]:
from scipy.integrate import solve_ivp

Consider two tanks (10 L and 20 L respectively) connected in series. Pure water enters the first tank at Q = 10 L/min, and the exit flow rate of the second tank is the same. At t = 0 the first tank contains $m_1$ = 1.5 kg of salt, which is then continuously flushed into the second tank. Determine when the maximum concentration of salt will be reached in the second tank, as well as the amount.

In other words, solve this system of ODEs with $ m_1(0)=1.5, \,\, m_2(0)=0 $:

\begin{align}
\frac{\mathrm{d}m_1}{\mathrm{d}t} &= -Q c_1 = -m_1 \\
\frac{\mathrm{d}m_2}{\mathrm{d}t} &= Q c_1 - Q c_2 = m_1 - 0.5 m_2
\end{align}

In [ ]:
# solve_ivp?

The help states:
```Python
solve_ivp(
    ['fun', 't_span', 'y0', "method='RK45'", 't_eval=None', 'dense_output=False', 'events=None', 'vectorized=False', '**options'],
)```

For `solve_ivp()` you must define your system of differential equations as a function (`fun`) such that the first input is the one-dimensional independent variable, followed by the state variables as a vector. Secondly, pass the beginning and ending time values as a tuple, followed by the initial conditions. The time points to be evaluated are determined automatically, or can be specified (try it with `t_eval=None`). The solution is given as nested lists.

In [ ]:
def diffs(t, var):
    m1, m2 = var
    dm1dt = -m1
    dm2dt = m1 - 0.5*m2
    return dm1dt, dm2dt

tspan = np.linspace(0, 10, 100)
y0 = [1.5, 0]

sol = solve_ivp(diffs, (0, 10), y0, t_eval=tspan)
# print(sol)
# print(sol.y)

In [ ]:
m1 = sol.y[0]
m2 = sol.y[1]
i_max = np.argmax(m2)
print(tspan[i_max], 'min')
print(m2[i_max], 'kg')

plt.plot(tspan, m1, label='$m_1$')
plt.plot(tspan, m2, label='$m_2$')
plt.plot(tspan[i_max], m2[i_max], '^')
plt.ylabel('Salt (kg)')
plt.xlabel('Time (min)')
plt.legend()
plt.show()

## Introduction to optimisation

Optimisation is a very complex mathematical science involving finding the best solution to a problem under certain constraints. The reader should consult the docs for more detailed examples and algorithms. 

Here, the [three-hump camel function](https://en.wikipedia.org/wiki/Test_functions_for_optimization) is tested; its global minimum is at (0,0). With a guess of (1,1) it only finds a local minimum.

In [ ]:
from scipy.optimize import fmin

In [ ]:
# 2 parameter function to be minimised, with an initial guess

def func(var):
    X, Y = var
    return 2*X**2 - 1.05*X**4 + X**6/6 + X*Y + Y**2

fmin(func, [1,1])

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline

In [ ]:
x = np.linspace(-2, 2, 13)
y = np.linspace(-2, 2, 13)
X, Y = np.meshgrid(x, y)
Z = 2*X**2 - 1.05*X**4 + X**6/6 + X*Y + Y**2

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='plasma', edgecolor='none')
ax.set_title('Three-hump camel')
plt.show()